## Bidding Architecture

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, StateGraph
import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import random
import json
import os

from IPython.display import Image, display

In [3]:
unique_id = "Market Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [4]:
from langsmith import Client

client = Client()

In [5]:
class PromptUpdate(BaseModel):
    """Review of the prompt"""
    updated_prompt: str = Field(description="Updated prompt based on the review")

class Bid(BaseModel):
    """Bid value for the prompt"""
    position: str = Field(description="Position of the expert providing the bid")
    bid: float = Field(description="Bid value for the prompt")

class ExpertAgent:
    """
    Expert Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, role: str, function: str, temp: float = 0.5, model: str = "gpt-4o"):
        self.position = position
        self.role = role
        self.function = function
        self.system_message = SystemMessage(content=f"""You are an expert: {self.position}. Your role: {self.role}. Your function: {self.function}.
You must use your expertise to guide all your thinking. You must speak only as an expert in your field.""")        
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )

    def bid(self, prompt: str, additional_info: str) -> float:
        """
        Bids on the prompt based on the expert's expertise.
        """
        template = """Your task is to bid on the following prompt: {prompt}.
The bid must reflect how much you believe you can improve the prompt.
You must bid based on your expertise and the prompt's requirements. 
The bid must be an integer between 0 and 10:
- 0 if you believe the prompt is perfect and cannot be improved.
- 10 if you believe the prompt is terrible and needs a lot of improvement.
You should be critical rather than generous in your bid. 
You will be awarded the chance to improve the prompt if your bid is the highest.

{additional_info}
Return only your position and bid value in JSON format below:

{{
    "position": "{position}",
    "bid": "Bid value",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=Bid)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "role", "function", "prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "role": self.role, "function": self.function, "prompt": prompt, "additional_info": additional_info})
                # Validate the output before returning
                if completion.position and completion.bid:
                    return completion
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")

    def generate(self, prompt: str, additional_info: str) -> PromptUpdate:
        """
        Generates an updated version of the prompt.
        """
        template = """Your task is to use your expertise to improve the below prompts effectiveness as an instruction for a large language model.
         
### Prompt: {prompt}.

Carefully review the prompt before making any changes.
Think about how you can utilise the skills that come with your position, role and function to write a prompt good at eliciting the desired response.

{additional_info}
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, do not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.
You will be heavily penalized if you do not follow these instructions.
The updated prompt must clearly instruct a large language model to generate high-quality content.

Return the updated prompt along with your review in JSON format below. You must not output your review.

{{
    "updated_prompt": "updated prompt based on the review"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptUpdate)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "role", "function", "prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "role": self.role, "function": self.function, "prompt": prompt, "additional_info": additional_info})
                # Validate the output before returning
                if completion.updated_prompt:
                    return completion
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")


class Experts(BaseModel):
    """Details of experts generated by the leader agent."""
    positions: List[str] = Field(description="List of positions of the experts")
    roles: List[str] = Field(description="List of roles of the experts")
    functions: List[str] = Field(description="List of functions of the experts")

class ModeratorAgent:
    """
    Moderator Agent class defining an agent that builds the experts and moderates the bidding process.
    """

    def __init__(self, base_prompt: str, additional_info: str = None, temp: float = 0.0, model: str = "gpt-4o", experts: List[ExpertAgent] = None):
        self.base_prompt = base_prompt
        self.prompt = base_prompt
        self.additional_info = additional_info
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.experts = experts if experts else self.generate_experts()
        self.prompt_history = {"base_prompt": self.base_prompt, "prompt_evolution": []}

    def generate_experts(self):
        """
        Generates a experts to help optimise prompts.
        """
        template = """Your task is to generate a team of three experts to optimise the prompt below.
    
### Prompt:  {base_prompt}

You must provide the positions, roles, and functions of the experts.
Be descriptive and detailed in your selection of positions, roles, and functions.
Below are the requirements for each field:
- Position: this must be analogous to a real-world job title.
- Role: this must be a description of the advisor's responsibilities in the context of prompt optimisation.
- Function: this must be a description of the advisor's function in the context of prompt optimisation.
Write the roles and functions as if they are a job description.
{additional_info}
Return only the positions, roles, and functions of the experts in JSON format below:

{{
    "positions": ["List of positions"],
    "roles": ["List of roles"],
    "functions": ["List of functions"]
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=Experts)
        prompt = PromptTemplate(
            template=template,
            input_variables=["base_prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"base_prompt": self.base_prompt, "additional_info": self.additional_info})
                if output.positions and output.roles and output.functions:
                    break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        experts = []
        positions, roles, functions = output.positions, output.roles, output.functions
        for position, role, function in zip(positions, roles, functions):
            experts.append(ExpertAgent(position, role, function))
        return experts

    def bidding_process(self, state: dict) -> dict:
        """
        Execution of bidding process to select the next expert to process the prompt.
        """
        # Collect bids from all experts
        bids = []
        for expert in self.experts:
            bid = expert.bid(state["prompt"], self.additional_info)
            bids.append(bid)
        # Sort bids in descending order
        bids.sort(key=lambda x: x.bid, reverse=True)
        for i in range(len(bids)):
            print("Position: {position}, Bid: {bid}".format(position=bids[i].position, bid=bids[i].bid))
        # If a tie occurs, randomly select a expert from the tied experts, else select the expert with the highest bid
        max_bid = max(bids, key=lambda x: x.bid).bid
        tied_experts = [expert for expert in bids if expert.bid == max_bid]
        if len(tied_experts) > 1:
            next_expert = tied_experts[random.randint(0, len(tied_experts) - 1)]
        else:
            next_expert = tied_experts[0]
        # Update the state with the next expert to process
        if max_bid <= 2.0:
            return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]}
        else:
            return {"next": next_expert.position, "prompt": state["prompt"], "messages": state["messages"]}
        
    def construct_expert_graph(self):
        """
        Constructs a graph of expert agents based on their roles and functions.
        """

        def agent_node(state, agent, position):
            try:
                result = agent.generate(state["prompt"], self.additional_info)
                updated_prompt = result.updated_prompt
                self.prompt = updated_prompt
                return {
                    "messages": state["messages"] + [AIMessage(content=f"Updated Prompt: {updated_prompt}", name=position)],
                    "prompt": updated_prompt,
                }
            except Exception as e:
                # Log the error and return to moderator with the most recent prompt
                print(f"Parsing failed for {position}: {e}")
                return {
                    "messages": state["messages"] + [AIMessage(content=f"Error: Parsing failed for {position} - {e}", name=position)],
                    "prompt": state["prompt"],
                    "next": "moderator"
                }
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            # The annotation tells the graph that new messages will always be added to the current states
            messages: Sequence[BaseMessage]
            next: str
            prompt: str

        workflow = StateGraph(AgentState)
        for expert in self.experts:
            # Create a node for each expert agent
            agent = ExpertAgent(expert.position, expert.role, expert.function)
            node = functools.partial(agent_node, agent=agent, position=expert.position)
            workflow.add_node(expert.position, node)
        workflow.add_node("moderator", self.bidding_process)

        members = [expert.position for expert in self.experts]
        for member in members:
            # We want our experts to ALWAYS "report back" to the moderator when done
            workflow.add_edge(member, "moderator")
        # The moderator populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("moderator", lambda x: x["next"], conditional_map)
        # Finally, add entrypoint
        workflow.set_entry_point("moderator")
        graph = workflow.compile()

        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of expert agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"Base Prompt: {self.base_prompt}", name="User")],
            "prompt": self.base_prompt,
            "next": "moderator"
        }

        # Construct the graph
        graph = self.construct_expert_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        # Run the graph
        for s in graph.stream(
            initial_state,
            {"recursion_limit": 50}
            ):
            if "__end__" not in s:
                # print(s)
                # print("----")
                continue

        # if not os.path.exists("prompt_history_market.json"):
        #     with open("prompt_history_market.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_market.json", "r") as f:
        #     data = json.load(f)
        #     data.append(self.prompt_history)
            
        # with open("prompt_history_market.json", "w") as f:
        #     json.dump(data, f, indent=4)
        
        return s

In [6]:
base_prompt = """Solve the following maths problem: {content}"""
additional_info = "The problem should only require elementary arithmetic operations. The output should be the answer only."

leader_agent = ModeratorAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)

# software_engineer = ExpertAgent(
#     position="Software Engineer", 
#     role="Design, develop, and maintain software applications",
#     function="Analyze requirements, create design specifications, write code, test software, and provide technical support"
#     )
# ai_prompt_engineer = ExpertAgent(
#     position="AI Prompt Engineer", 
#     role="Design, develop, and optimize prompts for AI models",
#     function="Craft effective prompts, test and evaluate AI responses, refine prompts for optimal performance, and ensure ethical considerations"
#     )
# software_architect = ExpertAgent(
#     position="Software Architect", 
#     role="Design and oversee the technical architecture of software systems",
#     function="Define architecture, select technologies, establish standards, provide technical leadership, and ensure quality and scalability"
#     )
# leader_agent.experts = [software_engineer, ai_prompt_engineer, software_architect]
for expert in leader_agent.experts:
    print("Position: ", expert.position + "\nRole: ", expert.role + "\nFunction: ", expert.function + "\n")

/Users/iwatson/Documents/Research Project/prompt-optimisation/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Position:  Prompt Engineer
Role:  The Prompt Engineer is responsible for designing and refining the prompt to ensure clarity and precision. They work on structuring the prompt in a way that maximizes the likelihood of obtaining the correct and concise answer.
Function:  The Prompt Engineer's function is to optimize the wording and structure of the prompt to make it as clear and effective as possible. They focus on eliminating any potential confusion and ensuring the prompt is straightforward.

Position:  Mathematics Content Specialist
Role:  The Mathematics Content Specialist is responsible for ensuring the mathematical accuracy and appropriateness of the problem. They review the content to confirm that it only requires elementary arithmetic operations and is suitable for the intended audience.
Function:  The Mathematics Content Specialist's function is to validate the mathematical content of the prompt. They ensure that the problem is solvable using only elementary arithmetic operatio

In [7]:
result = leader_agent.optimise_prompt()

Position: Quality Assurance Analyst, Bid: 5.0
Position: Prompt Engineer, Bid: 3.0
Position: Mathematics Content Specialist, Bid: 3.0
Position: Quality Assurance Analyst, Bid: 5.0
Position: Prompt Engineer, Bid: 4.0
Position: Mathematics Content Specialist, Bid: 4.0
Position: Quality Assurance Analyst, Bid: 4.0
Position: Prompt Engineer, Bid: 2.0
Position: Mathematics Content Specialist, Bid: 2.0
Position: Prompt Engineer, Bid: 2.0
Position: Mathematics Content Specialist, Bid: 2.0
Position: Quality Assurance Analyst, Bid: 1.0


In [8]:
result["moderator"]["prompt"]

'Solve the following elementary arithmetic problem: {content}. Provide only the final answer as a numerical value and ensure it is formatted correctly.'

### Concurrent Runs

In [11]:
base_prompt = "Classify the sentence as positive or negative: {content}"
additional_info = "This is a classification task with only two classes: positive and negative. The prompt should generalize."

moderator_agent_1 = ModeratorAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for expert in moderator_agent_1.experts:
    print("Position: ", expert.position + "\nRole: ", expert.role + "\nFunction: ", expert.function + "\n")

moderator_agent_2 = ModeratorAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for expert in moderator_agent_2.experts:
    print("Position: ", expert.position + "\nRole: ", expert.role + "\nFunction: ", expert.function + "\n")

moderator_agent_3 = ModeratorAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
for expert in moderator_agent_3.experts:
    print("Position: ", expert.position + "\nRole: ", expert.role + "\nFunction: ", expert.function + "\n")

Position:  Linguistic Analyst
Role:  The Linguistic Analyst is responsible for ensuring that the prompt is clear, concise, and unambiguous. They will analyze the language used in the prompt to ensure it is easily understandable and free of any potential biases.
Function:  The Linguistic Analyst will review and refine the wording of the prompt to ensure clarity and neutrality, preventing any language that could lead to misinterpretation or bias in classification.

Position:  Machine Learning Engineer
Role:  The Machine Learning Engineer is responsible for optimizing the prompt for machine learning models. They will ensure that the prompt is structured in a way that maximizes the model's ability to accurately classify the sentence as positive or negative.
Function:  The Machine Learning Engineer will test and iterate on the prompt structure to enhance the performance of classification algorithms, ensuring that the prompt facilitates accurate and reliable model outputs.

Position:  User E

KeyboardInterrupt: 

In [ ]:
import concurrent.futures

# Assuming leader_agent is already defined and initialized
def run_optimisation(agent: LeaderAgent):
    return agent.optimise_prompt()

# Run 3 concurrent instances
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(run_optimisation, agent) for agent in [leader_agent_1, leader_agent_2, leader_agent_3]]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)
    print("----")

class PromptMerge(BaseModel):
    """Merged prompt based on the best parts of each prompt."""
    final_prompt: str = Field(description="Result of merging prompts")

# OpenAI Agent to pull togther best parts of each result
def merge_results(results):
    """
    Agent to merge best parts of each prompt
    """
    llm = ChatOpenAI(
        temperature=1.0,
        model="gpt-4o",
    )
    system_message = """You are an experienced AI prompt engineer. Your role is to combine good prompts to create great prompts!
You have in-depth knowledge of large language models and prompt engineering best practices. Use knowledge at all times to guide your thinking."""

    template = """Your task is to merge the best parts of the prompts below to create a better prompt.
Carefully consider the strengths of each prompt and how they can be combined effectively whilst maintaining clarity and relevance.
You will be penalized if the prompt is repetitive, lacks clarity or is incoherent.
Aspects of the prompts to consider:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
Consider aspects of good prompts beyond those listed above.
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.

Prompts: {results}

Return only the next worker to process or 'FINISH' in JSON format below:

{{
    "final_prompt": "Result of merging prompts",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
    pydantic_parser = PydanticOutputParser(pydantic_object=PromptMerge)
    prompt_template = PromptTemplate(
        system_message=system_message,
        template=template,
        input_variables=["results"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    chain = prompt_template | llm | pydantic_parser
    for _ in range(3):
        try:
            output = chain.invoke({"results": results})
            break
        except Exception as e:
            print("Exception occurred:", e)
            continue
    return output.final_prompt

final_result = merge_results(results)

{'base_prompt': 'Classify the sentence as positive or negative: {content}', 'prompt_evolution': []}
{'base_prompt': 'Classify the sentence as positive or negative: {content}', 'prompt_evolution': [{'review': "The prompt should explicitly instruct the LLM to consider sentiment polarity, specify that the classes are only 'positive' or 'negative,' and ensure clarity in classification without introducing unnecessary complexities or ambiguities.", 'updated_prompt': "Classify the sentiment of the given sentence as either 'positive' or 'negative': {content}. Ensure the classification is based strictly on sentiment polarity."}]}
{'base_prompt': 'Classify the sentence as positive or negative: {content}', 'prompt_evolution': [{'review': 'The prompt needs clarity on the definition of positive and negative sentiment. Additionally, it should encourage consideration of context. Explicitly requiring the LLM to focus only on the sentiment will improve performance.', 'updated_prompt': 'Classify the sen

In [ ]:
print(final_result)

Classify the sentiment of the following sentence as either positive or negative: {content}. Evaluate tone and context carefully for precise classification. Ensure the classification is based strictly on sentiment polarity. Avoid neutrality. If sentiment is ambiguous, choose the more likely classification.
